In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.float_format', '{:.2f}'.format)  
pd.set_option('display.max_columns', None)            
pd.set_option('display.max_rows', 20)                  
pd.set_option('display.width', 150)

In [37]:
df = pd.read_csv('./Dataset/marketing_campaign_dataset.csv')

display(df.head())

df.info()

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,"$16,174.00",6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,"$11,566.00",5.61,New York,German,116,7523,7,Fashionistas,2021-01-02
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,"$10,200.00",7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,"$12,724.00",5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,"$16,452.00",6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       200000 non-null  int64  
 1   Company           200000 non-null  object 
 2   Campaign_Type     200000 non-null  object 
 3   Target_Audience   200000 non-null  object 
 4   Duration          200000 non-null  object 
 5   Channel_Used      200000 non-null  object 
 6   Conversion_Rate   200000 non-null  float64
 7   Acquisition_Cost  200000 non-null  object 
 8   ROI               200000 non-null  float64
 9   Location          200000 non-null  object 
 10  Language          200000 non-null  object 
 11  Clicks            200000 non-null  int64  
 12  Impressions       200000 non-null  int64  
 13  Engagement_Score  200000 non-null  int64  
 14  Customer_Segment  200000 non-null  object 
 15  Date              200000 non-null  object 
dtypes: float64(2), int64

In [38]:
# Xóa cột không cần thiết (ví dụ: Campaign_ID, Company)
df_clean = df.drop(columns=["Campaign_ID", "Company"])

In [39]:
df_clean["Acquisition_Cost"] = (
    df_clean["Acquisition_Cost"]
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)   # bỏ $ và ,
    .astype(float)
)


In [40]:
df_clean["CTR"] = df_clean["Clicks"] / df_clean["Impressions"]
df_clean["CPC"] = df_clean["Acquisition_Cost"] / df_clean["Clicks"]

display(df_clean.head())

,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,CTR,CPC
0,Email,Men 18-24,30 days,Google Ads,0.04,16174.00,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01,0.26,31.96
1,Email,Women 35-44,60 days,Google Ads,0.12,11566.00,5.61,New York,German,116,7523,7,Fashionistas,2021-01-02,0.02,99.71
2,Influencer,Men 25-34,30 days,YouTube,0.07,10200.00,7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03,0.08,17.47
3,Display,All Ages,60 days,YouTube,0.11,12724.00,5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04,0.12,58.64
4,Email,Men 25-34,15 days,YouTube,0.05,16452.00,6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05,0.09,43.41


In [41]:
# Xử lý dữ liệu thiếu
missing_summary = df_clean.isnull().sum()
print("\n===== Dữ liệu thiếu theo cột =====")
display(missing_summary)

# Điền giá trị thiếu (không dùng inplace để tránh cảnh báo)
df_clean["Conversion_Rate"] = df_clean["Conversion_Rate"].fillna(df_clean["Conversion_Rate"].mean())
df_clean["Customer_Segment"] = df_clean["Customer_Segment"].fillna("Unknown")

print("\n===== Kiểm tra lại dữ liệu thiếu =====")
display(df_clean.isnull().sum())


===== Dữ liệu thiếu theo cột =====


Campaign_Type       0
Target_Audience     0
Duration            0
Channel_Used        0
Conversion_Rate     0
Acquisition_Cost    0
ROI                 0
Location            0
Language            0
Clicks              0
Impressions         0
Engagement_Score    0
Customer_Segment    0
Date                0
CTR                 0
CPC                 0
dtype: int64


===== Kiểm tra lại dữ liệu thiếu =====


Campaign_Type       0
Target_Audience     0
Duration            0
Channel_Used        0
Conversion_Rate     0
Acquisition_Cost    0
ROI                 0
Location            0
Language            0
Clicks              0
Impressions         0
Engagement_Score    0
Customer_Segment    0
Date                0
CTR                 0
CPC                 0
dtype: int64

In [42]:
# Xử lý dữ liệu ngoại lai
# Cắt ngưỡng
df_clean["Acquisition_Cost_clipped"] = df_clean["Acquisition_Cost"].clip(upper=20000)
# Log-transform
df_clean["Acquisition_Cost_log"] = np.log1p(df_clean["Acquisition_Cost"])

display(df_clean.head())

,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,CTR,CPC,Acquisition_Cost_clipped,Acquisition_Cost_log
0,Email,Men 18-24,30 days,Google Ads,0.04,16174.00,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01,0.26,31.96,16174.00,9.69
1,Email,Women 35-44,60 days,Google Ads,0.12,11566.00,5.61,New York,German,116,7523,7,Fashionistas,2021-01-02,0.02,99.71,11566.00,9.36
2,Influencer,Men 25-34,30 days,YouTube,0.07,10200.00,7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03,0.08,17.47,10200.00,9.23
3,Display,All Ages,60 days,YouTube,0.11,12724.00,5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04,0.12,58.64,12724.00,9.45
4,Email,Men 25-34,15 days,YouTube,0.05,16452.00,6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05,0.09,43.41,16452.00,9.71


In [43]:
# Text
df_clean["Campaign_Type"] = df_clean["Campaign_Type"].str.lower()
df_clean["Target_Audience"] = df_clean["Target_Audience"].str.lower()

# Time
df_clean["Date"] = pd.to_datetime(df_clean["Date"])
df_clean["Month"] = df_clean["Date"].dt.month
df_clean["Year"] = df_clean["Date"].dt.year
df_clean["Is_Weekend"] = df_clean["Date"].dt.weekday >= 5  # 5,6 = Saturday, Sunday

print("\n===== 5 dòng đầu sau biến đổi nâng cao =====")
display(df_clean.head())


===== 5 dòng đầu sau biến đổi nâng cao =====


,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Language,Clicks,Impressions,Engagement_Score,Customer_Segment,Date,CTR,CPC,Acquisition_Cost_clipped,Acquisition_Cost_log,Month,Year,Is_Weekend
0,email,men 18-24,30 days,Google Ads,0.04,16174.00,6.29,Chicago,Spanish,506,1922,6,Health & Wellness,2021-01-01,0.26,31.96,16174.00,9.69,1,2021,False
1,email,women 35-44,60 days,Google Ads,0.12,11566.00,5.61,New York,German,116,7523,7,Fashionistas,2021-01-02,0.02,99.71,11566.00,9.36,1,2021,True
2,influencer,men 25-34,30 days,YouTube,0.07,10200.00,7.18,Los Angeles,French,584,7698,1,Outdoor Adventurers,2021-01-03,0.08,17.47,10200.00,9.23,1,2021,True
3,display,all ages,60 days,YouTube,0.11,12724.00,5.55,Miami,Mandarin,217,1820,7,Health & Wellness,2021-01-04,0.12,58.64,12724.00,9.45,1,2021,False
4,email,men 25-34,15 days,YouTube,0.05,16452.00,6.50,Los Angeles,Mandarin,379,4201,3,Health & Wellness,2021-01-05,0.09,43.41,16452.00,9.71,1,2021,False
